In [4]:
import re
import os
import csv

import matplotlib
matplotlib.use("Agg")  # for servers with no display
import matplotlib.pyplot as plt

# ====================================================
# 1. EDIT THIS: path to your MMDetection .log file
# ====================================================
LOG_PATH = "/data/sundeep/Point-Beyond-Class/Output/logs/mmdet-RSNA/exp2_faster512noBB_20p__train_LableBox_PseudoBox__exp1_stage1_data20p_baseline_lr02.log"
# ====================================================


def parse_log(log_path):
    """
    Parse an MMDetection plain .log file and extract metrics per validation epoch.
    Returns a list of dicts, one per epoch.
    """
    with open(log_path, "r") as f:
        lines = f.readlines()

    results = []

    # Epoch(val) line with bbox_mAP etc.
    epoch_re = re.compile(
        r"Epoch\(val\)\s*\[(\d+)\]\[(\d+)\]\s*bbox_mAP:\s*([0-9.\-]+),\s*"
        r"bbox_mAP_50:\s*([0-9.\-]+),\s*bbox_mAP_75:\s*([0-9.\-]+),\s*"
        r"bbox_mAP_s:\s*([0-9.\-]+),\s*bbox_mAP_m:\s*([0-9.\-]+),\s*bbox_mAP_l:\s*([0-9.\-]+)"
    )

    i = 0
    n = len(lines)
    while i < n:
        line = lines[i]
        m = epoch_re.search(line)
        if not m:
            i += 1
            continue

        # Basic bbox metrics from Epoch(val) line
        record = {
            "epoch": int(m.group(1)),
            "iter": int(m.group(2)),
            "bbox_mAP": float(m.group(3)),
            "bbox_mAP_50": float(m.group(4)),
            "bbox_mAP_75": float(m.group(5)),
            "bbox_mAP_s": float(m.group(6)),
            "bbox_mAP_m": float(m.group(7)),
            "bbox_mAP_l": float(m.group(8)),
        }

        # Now scan forward for COCO AP/AR lines belonging to this epoch
        ap_vals = []
        ar_vals = []
        j = i + 1
        while j < n:
            l = lines[j]
            # Stop when we hit the next Epoch(val)
            if "Epoch(val)" in l and epoch_re.search(l):
                break

            if "Average Precision" in l and "=" in l:
                try:
                    val = float(l.strip().split("=")[-1])
                    ap_vals.append(val)
                except ValueError:
                    pass
            elif "Average Recall" in l and "=" in l:
                try:
                    val = float(l.strip().split("=")[-1])
                    ar_vals.append(val)
                except ValueError:
                    pass
            j += 1

        # Map AP values (if present)
        if len(ap_vals) >= 6:
            record["COCO_AP_50_95"] = ap_vals[0]
            record["COCO_AP_50"] = ap_vals[1]
            record["COCO_AP_75"] = ap_vals[2]
            record["COCO_AP_s"] = ap_vals[3]
            record["COCO_AP_m"] = ap_vals[4]
            record["COCO_AP_l"] = ap_vals[5]

        # Map AR values (if present)
        if len(ar_vals) >= 6:
            record["COCO_AR_all_100"] = ar_vals[0]
            record["COCO_AR_all_300"] = ar_vals[1]
            record["COCO_AR_all_1000"] = ar_vals[2]
            record["COCO_AR_s"] = ar_vals[3]
            record["COCO_AR_m"] = ar_vals[4]
            record["COCO_AR_l"] = ar_vals[5]

        results.append(record)
        i = j  # continue from next block

    return results


def save_csv(records, csv_path):
    if not records:
        print("No records to save.")
        return
    keys = sorted(records[0].keys(), key=lambda k: (k != "epoch", k))
    with open(csv_path, "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=keys)
        writer.writeheader()
        for r in records:
            writer.writerow(r)
    print(f"Saved CSV summary to: {csv_path}")


def plot_ap_curves(records, out_path):
    epochs = [r["epoch"] for r in records]
    bbox_mAP = [r.get("bbox_mAP") for r in records]
    bbox_mAP_50 = [r.get("bbox_mAP_50") for r in records]
    bbox_mAP_75 = [r.get("bbox_mAP_75") for r in records]

    plt.figure()
    plt.plot(epochs, bbox_mAP, marker="o", label="bbox_mAP (0.50:0.95)")
    plt.plot(epochs, bbox_mAP_50, marker="o", label="bbox_mAP_50")
    plt.plot(epochs, bbox_mAP_75, marker="o", label="bbox_mAP_75")
    plt.xlabel("Epoch")
    plt.ylabel("AP")
    plt.title("BBox AP Metrics vs Epoch")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(out_path)
    plt.close()
    print(f"Saved AP curve plot to: {out_path}")


def plot_coco_ap_curves(records, out_path):
    # Only plot if COCO AP keys exist
    if "COCO_AP_50_95" not in records[0]:
        print("COCO AP metrics not found in records; skipping COCO AP plot.")
        return

    epochs = [r["epoch"] for r in records]
    ap_50_95 = [r.get("COCO_AP_50_95") for r in records]
    ap_50 = [r.get("COCO_AP_50") for r in records]
    ap_75 = [r.get("COCO_AP_75") for r in records]

    plt.figure()
    plt.plot(epochs, ap_50_95, marker="o", label="COCO_AP_50_95")
    plt.plot(epochs, ap_50, marker="o", label="COCO_AP_50")
    plt.plot(epochs, ap_75, marker="o", label="COCO_AP_75")
    plt.xlabel("Epoch")
    plt.ylabel("AP")
    plt.title("COCO AP Metrics vs Epoch")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(out_path)
    plt.close()
    print(f"Saved COCO AP curve plot to: {out_path}")


def main():
    if not os.path.isfile(LOG_PATH):
        raise FileNotFoundError(f"Log file not found: {LOG_PATH}")

    print(f"Parsing log file: {LOG_PATH}")
    records = parse_log(LOG_PATH)

    if not records:
        print("No validation epochs found! Check the log format.")
        return

    # Sort by epoch just in case
    records.sort(key=lambda r: r["epoch"])

    # Find best epoch by bbox_mAP_50
    best = max(records, key=lambda r: r.get("bbox_mAP_50", -1e9))
    print("\n========= BEST EPOCH (by bbox_mAP_50) =========")
    for k in sorted(best.keys()):
        print(f"{k:20s}: {best[k]}")
    print("===============================================\n")

    # Output paths (same folder as log)
    out_dir = os.path.dirname(LOG_PATH)
    csv_path = os.path.join(out_dir, "metrics_summary.csv")
    ap_plot_path = os.path.join(out_dir, "metrics_ap_curves.png")
    coco_ap_plot_path = os.path.join(out_dir, "metrics_coco_ap_curves.png")

    # Save CSV + plots
    save_csv(records, csv_path)
    plot_ap_curves(records, ap_plot_path)
    plot_coco_ap_curves(records, coco_ap_plot_path)


if __name__ == "__main__":
    main()

Parsing log file: /data/sundeep/Point-Beyond-Class/Output/logs/mmdet-RSNA/exp2_faster512noBB_20p__train_LableBox_PseudoBox__exp1_stage1_data20p_baseline_lr02.log

========= BEST EPOCH (by bbox_mAP_50) =========
COCO_AP_50          : 0.313
COCO_AP_50_95       : 0.076
COCO_AP_75          : 0.011
COCO_AP_l           : 0.077
COCO_AP_m           : 0.0
COCO_AP_s           : -1.0
COCO_AR_all_100     : 0.261
COCO_AR_all_1000    : 0.261
COCO_AR_all_300     : 0.261
COCO_AR_l           : 0.263
COCO_AR_m           : 0.039
COCO_AR_s           : -1.0
bbox_mAP            : 0.076
bbox_mAP_50         : 0.313
bbox_mAP_75         : 0.011
bbox_mAP_l          : 0.077
bbox_mAP_m          : 0.0
bbox_mAP_s          : -1.0
epoch               : 12
iter                : 301

Saved CSV summary to: /data/sundeep/Point-Beyond-Class/Output/logs/mmdet-RSNA/metrics_summary.csv
Saved AP curve plot to: /data/sundeep/Point-Beyond-Class/Output/logs/mmdet-RSNA/metrics_ap_curves.png
Saved COCO AP curve plot to: /data/sunde